In [377]:
import pandas as pd
import geopandas as gpd
from haversine import haversine
import numpy as np
import math
from datetime import timedelta
import itertools
from shapely.geometry import shape, Point
import random


# Parameters

In [378]:
num_samples = 1000  # Number of order samples
lambda_ = 5  # rate of occurence poisson distribution
long_mean = -46.64801  # long center sp
long_stddev = 0.05  # Standard deviation of longitude distribution
lat_mean = -23.65023  # lat center sp
lat_stddev = 0.05  # Standard deviation of the latitude distribution

# Generating timestamps following a Poisson distribution


In [379]:
# Generate random timestamps following a Poisson distribution
poisson_timestamps = np.random.poisson(lambda_, num_samples)

# Convert the Poisson timestamps to actual timestamps
start_timestamp = pd.Timestamp("2024-04-01 00:00:00")  # Define your start time
timestamps = [start_timestamp + pd.Timedelta(minutes=t) for t in np.cumsum(poisson_timestamps)]


In [380]:
timestamps

[Timestamp('2024-04-01 00:06:00'),
 Timestamp('2024-04-01 00:12:00'),
 Timestamp('2024-04-01 00:15:00'),
 Timestamp('2024-04-01 00:18:00'),
 Timestamp('2024-04-01 00:21:00'),
 Timestamp('2024-04-01 00:24:00'),
 Timestamp('2024-04-01 00:29:00'),
 Timestamp('2024-04-01 00:35:00'),
 Timestamp('2024-04-01 00:41:00'),
 Timestamp('2024-04-01 00:44:00'),
 Timestamp('2024-04-01 00:50:00'),
 Timestamp('2024-04-01 00:55:00'),
 Timestamp('2024-04-01 00:58:00'),
 Timestamp('2024-04-01 01:07:00'),
 Timestamp('2024-04-01 01:14:00'),
 Timestamp('2024-04-01 01:18:00'),
 Timestamp('2024-04-01 01:22:00'),
 Timestamp('2024-04-01 01:28:00'),
 Timestamp('2024-04-01 01:33:00'),
 Timestamp('2024-04-01 01:37:00'),
 Timestamp('2024-04-01 01:46:00'),
 Timestamp('2024-04-01 01:52:00'),
 Timestamp('2024-04-01 01:54:00'),
 Timestamp('2024-04-01 01:59:00'),
 Timestamp('2024-04-01 02:08:00'),
 Timestamp('2024-04-01 02:19:00'),
 Timestamp('2024-04-01 02:23:00'),
 Timestamp('2024-04-01 02:28:00'),
 Timestamp('2024-04-

# Generating timestamps following a Poisson Process


In [381]:
num_samples = 1000  # Number of order samples
lambda_ = 0.5  # rate of occurrence (lambda) for the Poisson process

# Generate random intervals between events following an exponential distribution
intervals = np.random.exponential(scale=1/lambda_, size=num_samples)

# Convert the intervals to timestamps starting from a given timestamp
start_timestamp = pd.Timestamp("2024-04-01 00:00:00")  # Define your start time
timestamps = [start_timestamp]
for interval in intervals:
    timestamps.append(timestamps[-1] + pd.Timedelta(minutes=interval))

# Remove the first element, which is the initial start time
timestamps = timestamps[1:]

# Now 'timestamps' represents the simulated timestamps of events in a Poisson process


In [382]:
timestamps

[Timestamp('2024-04-01 00:00:16.297054633'),
 Timestamp('2024-04-01 00:01:09.788653951'),
 Timestamp('2024-04-01 00:02:05.006835097'),
 Timestamp('2024-04-01 00:06:02.195468388'),
 Timestamp('2024-04-01 00:07:52.913574261'),
 Timestamp('2024-04-01 00:10:21.612201899'),
 Timestamp('2024-04-01 00:12:23.292662421'),
 Timestamp('2024-04-01 00:12:50.845930446'),
 Timestamp('2024-04-01 00:14:23.177186876'),
 Timestamp('2024-04-01 00:14:49.188136117'),
 Timestamp('2024-04-01 00:15:30.063378980'),
 Timestamp('2024-04-01 00:16:48.237860029'),
 Timestamp('2024-04-01 00:17:29.932081170'),
 Timestamp('2024-04-01 00:21:19.920923082'),
 Timestamp('2024-04-01 00:21:40.583208265'),
 Timestamp('2024-04-01 00:21:40.934968652'),
 Timestamp('2024-04-01 00:24:50.648048217'),
 Timestamp('2024-04-01 00:25:07.220934918'),
 Timestamp('2024-04-01 00:27:27.109002227'),
 Timestamp('2024-04-01 00:27:33.953761631'),
 Timestamp('2024-04-01 00:30:53.088636705'),
 Timestamp('2024-04-01 00:34:54.977794132'),
 Timestamp

# Reading city df and getting SP as test

In [383]:
df_muns = gpd.read_file(r"C:\Users\EduardoMontez\Downloads\municipios_2010-20240620T030303Z-001\municipios_2010")
df_sp = df_muns[df_muns['nome']=='São Paulo']

# Defining SP centroid
df_sp['centroid'] =df_sp.centroid
df_sp

C:\Users\EduardoMontez\AppData\Local\Temp\ipykernel_28552\1753475512.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_sp['centroid'] =df_sp.centroid
C:\Users\EduardoMontez\anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,id,nome,uf,populacao,pib,estado_id,codigo_ibg,geometry,centroid
5541,5345,São Paulo,SP,11253503,443600102,26,3550308,"POLYGON ((-46.53723 -23.36918, -46.54499 -23.3...",POINT (-46.64801 -23.65023)


In [384]:
def generate_normal_distribution(mean, std_dev, n):
    samples = np.random.normal(mean, std_dev, n)
    return samples.tolist()

# Generate samples for the first distribution
long_samples = generate_normal_distribution(long_mean, long_stddev, num_samples)

# Generate samples for the second distribution
lat_samples = generate_normal_distribution(lat_mean, lat_stddev, num_samples)

In [385]:
lat_samples

[-23.686948531552172,
 -23.628091945180184,
 -23.673768343249083,
 -23.65401196700561,
 -23.66193484817878,
 -23.656863714109125,
 -23.677563735486615,
 -23.714334830199732,
 -23.676562110276915,
 -23.719945822475648,
 -23.662369038764933,
 -23.705174579408652,
 -23.74982583918843,
 -23.694838508841364,
 -23.715027678781038,
 -23.59944163607045,
 -23.696925760103312,
 -23.700747803727047,
 -23.67941960496827,
 -23.685118960043887,
 -23.62832296346067,
 -23.637746918922982,
 -23.632996255396595,
 -23.685870764518977,
 -23.613579633635542,
 -23.682332977291434,
 -23.592471510921634,
 -23.686060556445195,
 -23.59738805462367,
 -23.58008637604531,
 -23.658936611355188,
 -23.65144342702816,
 -23.676403074036966,
 -23.695191754381387,
 -23.646012918719297,
 -23.779545948772263,
 -23.60740183234246,
 -23.6221632781036,
 -23.697341648512218,
 -23.7410783667864,
 -23.685109651089334,
 -23.729637562593147,
 -23.663786947450053,
 -23.687155113357523,
 -23.669617802230537,
 -23.646195843705122,
 -

# Creating df_orders

In [386]:
df_orders = pd.DataFrame(columns=["order_id"])
df_orders['order_id'] = [i for i in range(1, num_samples+1)]
df_orders['order_longitude'] = long_samples
df_orders['order_latitude'] = lat_samples
df_orders['ordered_at_brt'] = timestamps
df_orders

,order_id,order_longitude,order_latitude,ordered_at_brt
0,1,-46.647749,-23.686949,2024-04-01 00:00:16.297054633
1,2,-46.592114,-23.628092,2024-04-01 00:01:09.788653951
2,3,-46.589021,-23.673768,2024-04-01 00:02:05.006835097
3,4,-46.650906,-23.654012,2024-04-01 00:06:02.195468388
4,5,-46.638050,-23.661935,2024-04-01 00:07:52.913574261
...,...,...,...,...
995,996,-46.593777,-23.650683,2024-04-02 06:46:02.928973000
996,997,-46.672580,-23.602760,2024-04-02 06:46:06.179345552
997,998,-46.625902,-23.607187,2024-04-02 06:49:41.432890109
998,999,-46.631848,-23.630848,2024-04-02 06:54:30.213499345


# Generate Hub df

In [387]:
# Extract the geometry from the DataFrame
geometry = df_sp['geometry'].iloc[0]

# Define the bounding box of the geometry
min_x, min_y, max_x, max_y = geometry.bounds

# Method two to define hub location
#min_x, min_y, max_x, max_y = (-46.7726,-23.5413,-46.4986,-23.7456)


# Generate 20 random points within the geometry
latitude_values = []
longitude_values = []
for _ in range(20):
    while True:
        # Generate a random point within the bounding box
        random_point = Point(random.uniform(min_x, max_x), random.uniform(min_y, max_y))
        
        # Check if the random point is inside the geometry
        if geometry.contains(random_point):
            # Extract latitude and longitude values
            latitude_values.append(random_point.y)
            longitude_values.append(random_point.x)
            break


In [388]:
num_hubs = 15
# Generate samples for the first distribution
longitude_values = generate_normal_distribution(long_mean, long_stddev, num_hubs)

# Generate samples for the second distribution
latitude_values = generate_normal_distribution(lat_mean, lat_stddev, num_hubs)

In [389]:
longitude_values

[-46.699060481396586,
 -46.64350391544691,
 -46.66716788415331,
 -46.656295296743224,
 -46.62609802461857,
 -46.649938547238804,
 -46.65331728256161,
 -46.627482562021015,
 -46.629494949946725,
 -46.61305734979938,
 -46.71852997059447,
 -46.66541126816691,
 -46.73502294434862,
 -46.67431594891575,
 -46.687641049035825]

In [390]:
n_hubs = 15
unique_hubs =  pd.DataFrame(columns=["hub_id", "hub_longitude", "hub_latitude"])
unique_hubs['hub_id'] = [i for i in range(1, n_hubs+1)]
unique_hubs['hub_longitude'] = longitude_values
unique_hubs['hub_latitude'] = latitude_values
unique_hubs

,hub_id,hub_longitude,hub_latitude
0,1,-46.699060,-23.609462
1,2,-46.643504,-23.671544
2,3,-46.667168,-23.576433
3,4,-46.656295,-23.700613
4,5,-46.626098,-23.680242
5,6,-46.649939,-23.610411
6,7,-46.653317,-23.605308
7,8,-46.627483,-23.660124
8,9,-46.629495,-23.665556
9,10,-46.613057,-23.632128


In [335]:
# Distância de haversine
def calculate_distance(row):
    order_coords = (row['order_latitude'], row['order_longitude'])
    hub_coords = (row['hub_latitude'], row['hub_longitude'])
    return haversine(order_coords, hub_coords)

def calculate_distance_order_with_order(row):
    order_coords = (row['order_latitude'], row['order_longitude'])
    order2_coords = (row['order_latitude_open_order'], row['order_longitude_open_order'])
    return haversine(order_coords, order2_coords)

def assign_route_id(df,hub_col,minutes_wait, max_orders_per_route):
    # ordena por hub e horário
    df = df.sort_values(by=[hub_col, 'ordered_at_brt'])

    # inicializa contador de rotas e contagem de orders por rota
    current_route_id = 0
    orders_count_per_route = {}

    # incicializa dicionário para manter a par horário do último pedido para cada hub
    last_order_time = {}

    # Itera nas linhas
    for index, row in df.iterrows():
        hub_id = row[hub_col]
        ordered_at = row['ordered_at_brt']

        # Checa se é a primeira order do dia para o hub ou se o horário do pedido excede o max da rota
        if hub_id not in last_order_time or ordered_at > last_order_time[hub_id] + timedelta(minutes=minutes_wait):
            current_route_id += 1
            last_order_time[hub_id] = ordered_at
            orders_count_per_route[current_route_id] = 0

        # Checa se o max de orders por rota foi atingido
        if orders_count_per_route[current_route_id] >= max_orders_per_route:
            current_route_id += 1
            last_order_time[hub_id] = ordered_at
            orders_count_per_route[current_route_id] = 0

        # Faz a atribuição do route_id para aquela row
        df.at[index, 'route_id_moving_window'] = current_route_id
        orders_count_per_route[current_route_id] += 1

    return df.reset_index(drop=True)

def assign_route_id_fixed_window(df,hub_col,minutes_wait, max_orders_per_route):
    # ordena por hub e horário
    df = df.sort_values(by=[hub_col, 'ordered_at_brt'])

    # inicializa contador de rotas e contagem de orders por rota
    current_route_id = 0
    orders_count_per_route = {}

    # incicializa dicionário para manter a par horário do último pedido para cada hub
    last_route_time = {}

    # itera nas linhas
    for index, row in df.iterrows():
        hub_id = row[hub_col]
        ordered_at = row['ordered_at_brt']

        # Checa se é a primeira order do dia para o hub
        interval_start_time = ordered_at - timedelta(minutes=(ordered_at.minute % minutes_wait))

        # # Checa se é a primeira order do dia para o hub time ou se o tempo atual excede o limite da rota atual
        if hub_id not in last_route_time or ordered_at > last_route_time[hub_id]:
            current_route_id += 1
            last_route_time[hub_id] = interval_start_time + timedelta(minutes=minutes_wait)
            orders_count_per_route[current_route_id] = 0

        # Checa se o max numero de pedidos por rota foi atingido
        if orders_count_per_route[current_route_id] >= max_orders_per_route:
            current_route_id += 1
            last_route_time[hub_id] = interval_start_time + timedelta(minutes=minutes_wait)
            orders_count_per_route[current_route_id] = 0

        # Faz a atribuição do route_id para aquela row
        df.at[index, 'route_id_fixed_interval'] = current_route_id
        orders_count_per_route[current_route_id] += 1

    return df.reset_index(drop=True)


def new_assign_route_id(df,hub_col,minutes_wait, max_orders_per_route):
    # ordena por hub e horário
    df = df.sort_values(by=[hub_col, 'ordered_at_brt'])

    # inicializa contador de rotas e contagem de orders por rota
    current_route_id = 0
    orders_count_per_route = {}

    # incicializa dicionário para manter a par horário do último pedido para cada hub
    last_order_time = {}

    # Itera nas linhas
    for index, row in df.iterrows():
        hub_id = row[hub_col]
        ordered_at = row['ordered_at_brt']

        # Checa se é a primeira order do dia para o hub ou se o horário do pedido excede o max da rota
        if hub_id not in last_order_time or ordered_at > last_order_time[hub_id] + timedelta(minutes=minutes_wait):
            current_route_id += 1
            last_order_time[hub_id] = ordered_at
            orders_count_per_route[current_route_id] = 0

        # Checa se o max de orders por rota foi atingido
        if orders_count_per_route[current_route_id] >= max_orders_per_route:
            current_route_id += 1
            last_order_time[hub_id] = ordered_at
            orders_count_per_route[current_route_id] = 0

        # Faz a atribuição do route_id para aquela row
        df.at[index, 'route_id_new_model'] = current_route_id
        orders_count_per_route[current_route_id] += 1

    return df.reset_index(drop=True)

import itertools

def filter_one_route(df_routes,route_id_col,hub_id_col,n_routes,route_id_counter):
    route = df_routes[df_routes[route_id_col]==route_id_counter].reset_index(drop=True).rename(columns={'order_id':'id','order_latitude':'latitude','order_longitude':'longitude'})
    route['id'] = route['id'].astype(str)
    route[hub_id_col] = route[hub_id_col].astype(str)

    # New row to add
    new_row = {route_id_col: route[route_id_col][0],
               'id': route[hub_id_col][0],
               'latitude': route['hub_latitude'][0],
               'longitude': route['hub_longitude'][0],
               hub_id_col: route[hub_id_col][0],
               'hub_latitude': route['hub_latitude'][0],
               'hub_longitude': route['hub_longitude'][0],           
               }

    # Create a DataFrame from the new row dictionary
    new_row_pd = pd.DataFrame([new_row])

    # Concatenate the new row DataFrame with the existing DataFrame
    route = pd.concat([new_row_pd, route]).reset_index(drop=True)
    return route
    
def get_distance_matrix(route_df):
    # Cria matriz de distâncias com zeros
    n = len(route_df)
    dist_matrix = np.zeros((n, n))

    # Itera para montar a matriz quadrada de distâncias (todos com todos)
    for i in range(n):
        for j in range(n):
            dist_matrix[i][j] = haversine((route_df['latitude'][i],route_df['longitude'][i]),
                                           (route_df['latitude'][j],route_df['longitude'][j]))

    return dist_matrix


def solve_tsp_nearest(distances):
    num_cities = len(distances)
    visited = [False] * num_cities
    tour = []
    total_distance = 0
    partial_distances = []  # List to store partial distances
    
    # Start at the first city
    current_city = 0
    tour.append(current_city)
    visited[current_city] = True
    
    # Repeat until all cities have been visited
    while len(tour) < num_cities:
        nearest_city = None
        nearest_distance = math.inf

        # Find the nearest unvisited city
        for city in range(num_cities):
            if not visited[city]:
                distance = distances[current_city][city]
                if distance < nearest_distance:
                    nearest_city = city
                    nearest_distance = distance
        
        # Move to the nearest city
        current_city = nearest_city
        tour.append(current_city)
        visited[current_city] = True
        total_distance += nearest_distance
        partial_distances.append(nearest_distance)  # Store the partial distance
    
    return tour, total_distance, partial_distances


def solve_tsp_brute_force_without_return(distances):
    num_cities = len(distances)
    cities = list(range(num_cities))
    min_distance = float('inf')
    optimal_tour = None
    partial_distances = []
    
    # Generate all permutations of cities
    for permutation in itertools.permutations(cities):
        tour_distance = 0
        partial_distance = []
        for i in range(num_cities - 1):
            tour_distance += distances[permutation[i]][permutation[i+1]]
            partial_distance.append(distances[permutation[i]][permutation[i+1]])
        
        # Check if the current permutation gives a shorter tour
        if tour_distance < min_distance:
            min_distance = tour_distance
            optimal_tour = list(permutation)
            partial_distances = partial_distance
        
    return optimal_tour, min_distance, partial_distances

def get_route_reindexed_df(route_df,route_id_col,tour,total_distances,partial_distances):
    route_reindex = route_df.reindex(index=route_df.index[tour]).drop([0]).reset_index(drop=True).rename(columns={'id':'order_id'})
    route_reindex['order_id_delivery_order'] = route_reindex.index+1
    route_reindex['route_total_distance'] = total_distance
    route_reindex['order_partial_distance'] = partial_distances
    route_reindex_final = route_reindex[[route_id_col,'order_id','order_id_delivery_order','route_total_distance','order_partial_distance']]
    return route_reindex_final

# Cross join orders and hubs

In [336]:
# Cross join dos DataFrames
df_final_order_hub = pd.merge(df_orders.assign(key=1), unique_hubs.assign(key=1), on='key').drop('key', axis=1)
df_final_order_hub

,order_id,order_longitude,order_latitude,ordered_at_brt,hub_id,hub_longitude,hub_latitude
0,1,-46.599077,-23.645623,2024-04-01 00:04:19.274215565,1,-46.690592,-23.702916
1,1,-46.599077,-23.645623,2024-04-01 00:04:19.274215565,2,-46.644221,-23.709041
2,1,-46.599077,-23.645623,2024-04-01 00:04:19.274215565,3,-46.596567,-23.625197
3,1,-46.599077,-23.645623,2024-04-01 00:04:19.274215565,4,-46.576244,-23.663943
4,1,-46.599077,-23.645623,2024-04-01 00:04:19.274215565,5,-46.663487,-23.610362
...,...,...,...,...,...,...,...
14995,1000,-46.564849,-23.663368,2024-04-02 09:55:28.225287958,11,-46.639909,-23.620135
14996,1000,-46.564849,-23.663368,2024-04-02 09:55:28.225287958,12,-46.629898,-23.589774
14997,1000,-46.564849,-23.663368,2024-04-02 09:55:28.225287958,13,-46.664658,-23.643593
14998,1000,-46.564849,-23.663368,2024-04-02 09:55:28.225287958,14,-46.775251,-23.679632


In [337]:
df_final_order_hub['distance'] = df_final_order_hub.apply(calculate_distance, axis=1).round(2)
df_final_order_hub = df_final_order_hub.drop_duplicates(subset=['order_id','hub_id'])
df_final_order_hub

,order_id,order_longitude,order_latitude,ordered_at_brt,hub_id,hub_longitude,hub_latitude,distance
0,1,-46.599077,-23.645623,2024-04-01 00:04:19.274215565,1,-46.690592,-23.702916,11.29
1,1,-46.599077,-23.645623,2024-04-01 00:04:19.274215565,2,-46.644221,-23.709041,8.42
2,1,-46.599077,-23.645623,2024-04-01 00:04:19.274215565,3,-46.596567,-23.625197,2.29
3,1,-46.599077,-23.645623,2024-04-01 00:04:19.274215565,4,-46.576244,-23.663943,3.09
4,1,-46.599077,-23.645623,2024-04-01 00:04:19.274215565,5,-46.663487,-23.610362,7.64
...,...,...,...,...,...,...,...,...
14995,1000,-46.564849,-23.663368,2024-04-02 09:55:28.225287958,11,-46.639909,-23.620135,9.03
14996,1000,-46.564849,-23.663368,2024-04-02 09:55:28.225287958,12,-46.629898,-23.589774,10.53
14997,1000,-46.564849,-23.663368,2024-04-02 09:55:28.225287958,13,-46.664658,-23.643593,10.40
14998,1000,-46.564849,-23.663368,2024-04-02 09:55:28.225287958,14,-46.775251,-23.679632,21.50


# Allocating based on nearest hub

In [338]:
sorted_result = df_final_order_hub.sort_values(by=['distance','hub_id'],ascending=[True, False])
top_hubs = sorted_result.groupby(['order_id']).head(1).reset_index(drop=True)
top_hubs

,order_id,order_longitude,order_latitude,ordered_at_brt,hub_id,hub_longitude,hub_latitude,distance
0,261,-46.639555,-23.620032,2024-04-01 08:52:49.750042260,11,-46.639909,-23.620135,0.04
1,287,-46.664478,-23.609966,2024-04-01 09:42:03.292020505,5,-46.663487,-23.610362,0.11
2,871,-46.715919,-23.623067,2024-04-02 05:36:34.135683421,6,-46.715423,-23.621975,0.13
3,398,-46.663062,-23.643433,2024-04-01 13:16:39.426108459,13,-46.664658,-23.643593,0.16
4,86,-46.663888,-23.645183,2024-04-01 03:10:36.310991811,13,-46.664658,-23.643593,0.19
...,...,...,...,...,...,...,...,...
995,567,-46.612071,-23.813880,2024-04-01 18:48:55.496742729,7,-46.608064,-23.713985,11.12
996,87,-46.702050,-23.804171,2024-04-01 03:11:52.437825077,1,-46.690592,-23.702916,11.32
997,453,-46.755544,-23.524583,2024-04-01 15:26:13.475672439,6,-46.715423,-23.621975,11.58
998,83,-46.595774,-23.825792,2024-04-01 03:05:42.476053803,7,-46.608064,-23.713985,12.50


# Assigning both fixed window and moving window

In [339]:
hubs_with_moving_route =assign_route_id(top_hubs,'hub_id',30,4) 
hubs_with_both_routes = assign_route_id_fixed_window(hubs_with_moving_route,'hub_id',30,4)
hubs_with_both_routes

,order_id,order_longitude,order_latitude,ordered_at_brt,hub_id,hub_longitude,hub_latitude,distance,route_id_moving_window,route_id_fixed_interval
0,4,-46.718602,-23.695621,2024-04-01 00:06:39.311488463,1,-46.690592,-23.702916,2.97,1.0,1.0
1,9,-46.726169,-23.691987,2024-04-01 00:14:25.573482729,1,-46.690592,-23.702916,3.82,1.0,1.0
2,36,-46.740537,-23.730637,2024-04-01 01:18:32.589737090,1,-46.690592,-23.702916,5.95,2.0,2.0
3,43,-46.693407,-23.728043,2024-04-01 01:35:09.629815740,1,-46.690592,-23.702916,2.81,2.0,3.0
4,53,-46.686270,-23.714534,2024-04-01 02:03:21.638165969,1,-46.690592,-23.702916,1.36,3.0,4.0
...,...,...,...,...,...,...,...,...,...,...
995,894,-46.714932,-23.644153,2024-04-02 06:18:28.630844171,15,-46.704035,-23.662853,2.36,480.0,595.0
996,895,-46.701347,-23.640940,2024-04-02 06:18:36.772386743,15,-46.704035,-23.662853,2.45,480.0,595.0
997,906,-46.710908,-23.643755,2024-04-02 06:41:34.302114117,15,-46.704035,-23.662853,2.24,481.0,596.0
998,954,-46.707899,-23.674768,2024-04-02 08:08:58.442976286,15,-46.704035,-23.662853,1.38,482.0,597.0


In [340]:
hubs_with_both_routes['route_id_moving_window'].nunique(),hubs_with_both_routes['route_id_fixed_interval'].nunique()

(483, 598)

In [341]:
hubs_with_both_routes.head(20)

,order_id,order_longitude,order_latitude,ordered_at_brt,hub_id,hub_longitude,hub_latitude,distance,route_id_moving_window,route_id_fixed_interval
0,4,-46.718602,-23.695621,2024-04-01 00:06:39.311488463,1,-46.690592,-23.702916,2.97,1.0,1.0
1,9,-46.726169,-23.691987,2024-04-01 00:14:25.573482729,1,-46.690592,-23.702916,3.82,1.0,1.0
2,36,-46.740537,-23.730637,2024-04-01 01:18:32.589737090,1,-46.690592,-23.702916,5.95,2.0,2.0
3,43,-46.693407,-23.728043,2024-04-01 01:35:09.629815740,1,-46.690592,-23.702916,2.81,2.0,3.0
4,53,-46.686270,-23.714534,2024-04-01 02:03:21.638165969,1,-46.690592,-23.702916,1.36,3.0,4.0
5,64,-46.708607,-23.692116,2024-04-01 02:26:22.573194074,1,-46.690592,-23.702916,2.19,3.0,4.0
6,72,-46.689559,-23.738424,2024-04-01 02:49:23.210942251,1,-46.690592,-23.702916,3.95,4.0,5.0
7,75,-46.691091,-23.730920,2024-04-01 02:52:37.023553137,1,-46.690592,-23.702916,3.11,4.0,5.0
8,82,-46.695449,-23.705343,2024-04-01 03:05:14.483976405,1,-46.690592,-23.702916,0.56,4.0,6.0
9,87,-46.702050,-23.804171,2024-04-01 03:11:52.437825077,1,-46.690592,-23.702916,11.32,4.0,6.0


# Allocating based on open routes

In [342]:
sample_pedidos = sorted_result.sort_values(by=['ordered_at_brt']).drop_duplicates(subset=['order_id']).tail(50)[['order_id','order_latitude','order_longitude','ordered_at_brt']].drop_duplicates(subset=['order_id']).sort_values(by=['ordered_at_brt']).reset_index(drop=True)
sample_pedidos

,order_id,order_latitude,order_longitude,ordered_at_brt
0,951,-23.644372,-46.586226,2024-04-02 08:05:03.380183458
1,952,-23.617639,-46.650510,2024-04-02 08:05:40.052810375
2,953,-23.666749,-46.629082,2024-04-02 08:08:19.057426977
3,954,-23.674768,-46.707899,2024-04-02 08:08:58.442976286
4,955,-23.634046,-46.716472,2024-04-02 08:09:15.506438884
5,956,-23.689334,-46.620924,2024-04-02 08:12:40.171290215
6,957,-23.610786,-46.620751,2024-04-02 08:14:50.523299800
7,958,-23.655159,-46.633368,2024-04-02 08:15:05.611096285
8,959,-23.683666,-46.651173,2024-04-02 08:15:51.688171776
9,960,-23.623579,-46.694758,2024-04-02 08:16:13.198610017


In [343]:
df_pedidos = sorted_result.sort_values(by=['ordered_at_brt']).drop_duplicates(subset=['order_id'])[['order_id','order_latitude','order_longitude','ordered_at_brt']].drop_duplicates(subset=['order_id']).sort_values(by=['ordered_at_brt']).reset_index(drop=True)
df = df_pedidos

#df = sample_pedidos

In [344]:
km_correct = 2
minutes_wait = 30
order_allocated_hub = pd.DataFrame(columns=['route_id','order_id','hub_id'])
open_routes = pd.DataFrame(columns=['route_id','order_id_waiting','order_latitude_open_order','order_longitude_open_order','hub_id_waiting','waiting_ordered_at_brt','max_time_order','max_time_route'])
route_id_count = 0

In [345]:
open_routes

,route_id,order_id_waiting,order_latitude_open_order,order_longitude_open_order,hub_id_waiting,waiting_ordered_at_brt,max_time_order,max_time_route


In [346]:
for i in range(len(df)):
    print(df.order_id[i])
    print(df[df.index==i])
    time_of_current_order = df[df.index==i]['ordered_at_brt'][i]
    open_routes['route_id_orders'] = open_routes.groupby(['route_id'])['order_id_waiting'].transform('count')    
    open_routes['max_time_route'] = open_routes.groupby(['order_id_waiting', 'route_id'])['max_time_route'].transform('min')
    open_routes = open_routes[(open_routes['max_time_route']>time_of_current_order)&(open_routes['route_id_orders']<4)] # São os que de fato ainda estão em aberto e as rotas não fechadas
    print(time_of_current_order)    
    if not open_routes.empty:
        print('tem rota aberta')
        pedido = df[df.index==i]
        orders_hub = open_routes.groupby(['hub_id_waiting']).agg(orders_hub_waiting=('order_id_waiting','count')).reset_index(drop=False)    
        distances_order_hub_df = pd.merge(pedido.assign(key=1), unique_hubs.assign(key=1), on='key').drop('key', axis=1)
        distances_order_hub_df['distance'] = distances_order_hub_df.apply(calculate_distance, axis=1)
        distances_order_hub_df_filtered =distances_order_hub_df.merge(orders_hub,left_on='hub_id',right_on='hub_id_waiting',how='left').fillna(0).sort_values(by=['distance']).reset_index(drop=True)
        distances_order_order_df = pd.merge(pedido.assign(key=1), open_routes.assign(key=1), on='key').drop('key', axis=1)
        distances_order_order_df['distance_order_to_order'] = distances_order_order_df.apply(calculate_distance_order_with_order, axis=1)
        distance_order_to_order = distances_order_order_df.merge(orders_hub,on='hub_id_waiting',how='left').fillna(0)
        distance_order_to_order_filtered = distance_order_to_order[distance_order_to_order['orders_hub_waiting']<4].sort_values(by=['distance_order_to_order']).reset_index(drop=True)
        
        print('dist order to order é ',distance_order_to_order_filtered['distance_order_to_order'][0],'hub em questão',distance_order_to_order_filtered['hub_id_waiting'][0])
        print('dist order to hub é ',distances_order_hub_df_filtered['distance'][0],'hub em questão',distances_order_hub_df_filtered['hub_id'][0])        
        if distance_order_to_order_filtered['distance_order_to_order'][0] <= distances_order_hub_df_filtered['distance'][0]+km_correct: # vamos alocar em uma rota existente
            print('distancia pro pedido menor que distancia pro hub +',km_correct,"km")
            print('hub alocado',(distance_order_to_order_filtered['hub_id_waiting'][0]),2)
            chosen_route = distance_order_to_order_filtered['route_id'][0]
            print('route_id',chosen_route)
            new_row_open_routes = [{'route_id':chosen_route,
                                    'order_id_waiting':distances_order_hub_df_filtered['order_id'][0],
                                    'order_latitude_open_order':distances_order_hub_df_filtered['order_latitude'][0],
                                    'order_longitude_open_order':distances_order_hub_df_filtered['order_longitude'][0],
                                    'hub_id_waiting':distance_order_to_order_filtered['hub_id_waiting'][0],
                                    'waiting_ordered_at_brt':distances_order_hub_df_filtered['ordered_at_brt'][0],
                                    'max_time_order':distances_order_hub_df_filtered['ordered_at_brt'][0]+timedelta(minutes=minutes_wait),
                                    'max_time_route':distance_order_to_order_filtered['max_time_route'][0]}]
            open_routes = pd.concat([open_routes, pd.DataFrame(new_row_open_routes)], ignore_index=True)
            new_row_order_allocated_hub = [{'route_id':chosen_route,'order_id': distances_order_hub_df_filtered['order_id'][0], 'hub_id': distance_order_to_order_filtered['hub_id_waiting'][0]}]
            order_allocated_hub = pd.concat([order_allocated_hub, pd.DataFrame(new_row_order_allocated_hub)], ignore_index=True)

        else:
            print('distancia pro pedido maior que distancia pro hub +',km_correct,"km")
            print('hub alocado',distances_order_hub_df_filtered['hub_id'][0])            
            print('hub_order',distance_order_to_order_filtered['hub_id_waiting'][0]) 
            hub_allocated = distances_order_hub_df_filtered['hub_id'][0]
            # Existe rota ja aberta nesse hub?
            try:
                print('passou aqui1')
                chosen_route = open_routes[open_routes.hub_id_waiting==hub_allocated]['route_id'].reset_index(drop=True)[0]
                print('rota em aberto, route_id: ',chosen_route)
                print('rota_escolhida:',chosen_route)
                print('orders no hub:',len(open_routes[open_routes.route_id==chosen_route]))
                if len(open_routes[open_routes.route_id==chosen_route])>=4: #inicia nova rota
                    print('passou aqui2')
                    chosen_route = max(order_allocated_hub['route_id']) + 1 
                    route_id_count = max(order_allocated_hub['route_id']) + 1
                    print('route_id',existing_route)
                    print('caiu aqui')
                    new_row_open_routes = [{'route_id':chosen_route,
                                            'order_id_waiting':distances_order_hub_df_filtered['order_id'][0] ,
                                            'order_latitude_open_order':distances_order_hub_df_filtered['order_latitude'][0] ,
                                            'order_longitude_open_order':distances_order_hub_df_filtered['order_longitude'][0],
                                            'hub_id_waiting':distances_order_hub_df_filtered['hub_id'][0],
                                            'waiting_ordered_at_brt':distances_order_hub_df_filtered['ordered_at_brt'][0],
                                            'max_time_order':distances_order_hub_df_filtered['ordered_at_brt'][0]+timedelta(minutes=minutes_wait),
                                            'max_time_route':distances_order_hub_df_filtered['ordered_at_brt'][0]+timedelta(minutes=minutes_wait)}]
                    open_routes = pd.concat([open_routes, pd.DataFrame(new_row_open_routes)], ignore_index=True)
                    new_row_order_allocated_hub = [{'route_id':chosen_route,'order_id': distances_order_hub_df_filtered['order_id'][0], 'hub_id':distances_order_hub_df_filtered['hub_id'][0] }]
                    order_allocated_hub = pd.concat([order_allocated_hub, pd.DataFrame(new_row_order_allocated_hub)], ignore_index=True)
                else:
                    print('passou aqui3')
                    new_row_open_routes = [{'route_id':chosen_route,
                                            'order_id_waiting':distances_order_hub_df_filtered['order_id'][0] ,
                                            'order_latitude_open_order':distances_order_hub_df_filtered['order_latitude'][0] ,
                                            'order_longitude_open_order':distances_order_hub_df_filtered['order_longitude'][0],
                                            'hub_id_waiting':distances_order_hub_df_filtered['hub_id'][0],
                                            'waiting_ordered_at_brt':distances_order_hub_df_filtered['ordered_at_brt'][0],
                                            'max_time_order':distances_order_hub_df_filtered['ordered_at_brt'][0]+timedelta(minutes=minutes_wait),
                                            'max_time_route':open_routes[open_routes['route_id']==chosen_route].reset_index()['max_time_route'][0]}]
                    open_routes = pd.concat([open_routes, pd.DataFrame(new_row_open_routes)], ignore_index=True)
                    new_row_order_allocated_hub = [{'route_id':chosen_route,'order_id': distances_order_hub_df_filtered['order_id'][0], 'hub_id':distances_order_hub_df_filtered['hub_id'][0]}]
                    order_allocated_hub = pd.concat([order_allocated_hub, pd.DataFrame(new_row_order_allocated_hub)], ignore_index=True)
                    print('passou aqui4')
            except: #Nãio tem rota aberta nesse pdv
                print('passou aqui5')
                route_id_count = max(order_allocated_hub['route_id']) + 1
                print('route_id',route_id_count)
                print('caiu aqui tb')
                new_row_open_routes = [{'route_id':route_id_count, #iniciando nova rota
                                        'order_id_waiting':distances_order_hub_df_filtered['order_id'][0] ,
                                        'order_latitude_open_order':distances_order_hub_df_filtered['order_latitude'][0] ,
                                        'order_longitude_open_order':distances_order_hub_df_filtered['order_longitude'][0],
                                        'hub_id_waiting':distances_order_hub_df_filtered['hub_id'][0],
                                        'waiting_ordered_at_brt':distances_order_hub_df_filtered['ordered_at_brt'][0],
                                        'max_time_order':distances_order_hub_df_filtered['ordered_at_brt'][0]+timedelta(minutes=minutes_wait),
                                        'max_time_route':distance_order_to_order_filtered['ordered_at_brt'][0]+timedelta(minutes=minutes_wait)}]
                open_routes = pd.concat([open_routes, pd.DataFrame(new_row_open_routes)], ignore_index=True)
                new_row_order_allocated_hub = [{'route_id':route_id_count,'order_id': distances_order_hub_df_filtered['order_id'][0], 'hub_id':distances_order_hub_df_filtered['hub_id'][0] }]
                order_allocated_hub = pd.concat([order_allocated_hub, pd.DataFrame(new_row_order_allocated_hub)], ignore_index=True)

    else:
        if i==0:
            route_id_count = route_id_count + 1
        else:            
            route_id_count = max(order_allocated_hub['route_id']) + 1
        print('não tem rota aberta')
        print('route_id',route_id_count)
        pedido = df[df.index==i]
        distances_order_hub_df = pd.merge(pedido.assign(key=1), unique_hubs.assign(key=1), on='key').drop('key', axis=1)
        distances_order_hub_df['distance'] = distances_order_hub_df.apply(calculate_distance, axis=1)
        distances_order_hub_df = distances_order_hub_df.sort_values(by=['distance']).reset_index(drop=True)
        print('dist pro hub mais próximo é',round(distances_order_hub_df['distance'][0],2))
        print('hub alocado',distances_order_hub_df['hub_id'][0])
        new_row_open_routes = [{'route_id':route_id_count,
                                'order_id_waiting':distances_order_hub_df['order_id'][0] ,
                                'order_latitude_open_order':distances_order_hub_df['order_latitude'][0] ,
                                'order_longitude_open_order':distances_order_hub_df['order_longitude'][0],
                                'hub_id_waiting':distances_order_hub_df['hub_id'][0],
                                'waiting_ordered_at_brt':distances_order_hub_df['ordered_at_brt'][0],
                                'max_time_order':distances_order_hub_df['ordered_at_brt'][0]+timedelta(minutes=minutes_wait),
                                'max_time_route':distances_order_hub_df['ordered_at_brt'][0]+timedelta(minutes=minutes_wait)},                              ]
        open_routes = pd.concat([open_routes, pd.DataFrame(new_row_open_routes)], ignore_index=True)
        new_row_order_allocated_hub = [{'route_id':route_id_count,'order_id': distances_order_hub_df['order_id'][0], 'hub_id':distances_order_hub_df['hub_id'][0] }]
        order_allocated_hub = pd.concat([order_allocated_hub, pd.DataFrame(new_row_order_allocated_hub)], ignore_index=True)

1
   order_id  order_latitude  order_longitude                ordered_at_brt
0         1      -23.645623       -46.599077 2024-04-01 00:04:19.274215565
2024-04-01 00:04:19.274215565
não tem rota aberta
route_id 1
dist pro hub mais próximo é 2.29
hub alocado 3
2
   order_id  order_latitude  order_longitude                ordered_at_brt
1         2       -23.76564       -46.660459 2024-04-01 00:04:22.760417702
2024-04-01 00:04:22.760417702
tem rota aberta
dist order to order é  14.73614026216229 hub em questão 3
dist order to hub é  6.506984040327739 hub em questão 2
distancia pro pedido maior que distancia pro hub + 2 km
hub alocado 2
hub_order 3
passou aqui1
passou aqui5
route_id 2
caiu aqui tb
3
   order_id  order_latitude  order_longitude                ordered_at_brt
2         3      -23.545739        -46.63406 2024-04-01 00:06:21.650102260
2024-04-01 00:06:21.650102260
tem rota aberta
dist order to order é  11.66471050649936 hub em questão 3
dist order to hub é  4.914826013140017 h

C:\Users\EduardoMontez\AppData\Local\Temp\ipykernel_28552\4238206233.py:121: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  open_routes = pd.concat([open_routes, pd.DataFrame(new_row_open_routes)], ignore_index=True)
C:\Users\EduardoMontez\AppData\Local\Temp\ipykernel_28552\4238206233.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  distance_order_to_order = distances_order_order_df.merge(orders_hub,on='hub_id_waiting',how='left').fillna(0)
C:\Users\EduardoMontez\AppData\Local\Temp\ipykernel_28552\4238206233.py:18: Future

dist order to order é  5.170983270937098 hub em questão 2
dist order to hub é  1.8916842614605762 hub em questão 2
distancia pro pedido maior que distancia pro hub + 2 km
hub alocado 2
hub_order 2
passou aqui1
rota em aberto, route_id:  2
rota_escolhida: 2
orders no hub: 1
passou aqui3
passou aqui4
7
   order_id  order_latitude  order_longitude                ordered_at_brt
6         7      -23.680416       -46.717804 2024-04-01 00:11:10.833061239
2024-04-01 00:11:10.833061239
tem rota aberta
dist order to order é  1.6926314881869653 hub em questão 1
dist order to hub é  2.4042025205974813 hub em questão 15
distancia pro pedido menor que distancia pro hub + 2 km
hub alocado 1 2
route_id 4
8
   order_id  order_latitude  order_longitude                ordered_at_brt
7         8       -23.68724       -46.577901 2024-04-01 00:13:53.380263784
2024-04-01 00:13:53.380263784
tem rota aberta
dist order to order é  5.105418617230541 hub em questão 3
dist order to hub é  2.595955634984271 hub em 

C:\Users\EduardoMontez\AppData\Local\Temp\ipykernel_28552\4238206233.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  distance_order_to_order = distances_order_order_df.merge(orders_hub,on='hub_id_waiting',how='left').fillna(0)
C:\Users\EduardoMontez\AppData\Local\Temp\ipykernel_28552\4238206233.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  distance_order_to_order = distances_order_order_df.merge(orders_hub,on='hub_id_waiting',how='left').fillna(0)
C:\Users\EduardoMontez\AppData\Local\Temp\ipykernel_28552\4238206233.py:18: FutureWarning: Downc

dist order to order é  0.4753156191667477 hub em questão 12
dist order to hub é  3.455750046741381 hub em questão 12
distancia pro pedido menor que distancia pro hub + 2 km
hub alocado 12 2
route_id 3
13
    order_id  order_latitude  order_longitude                ordered_at_brt
12        13      -23.616648       -46.643257 2024-04-01 00:28:39.875361470
2024-04-01 00:28:39.875361470
tem rota aberta
dist order to order é  5.535071138435779 hub em questão 3
dist order to hub é  0.5163614179879614 hub em questão 11
distancia pro pedido maior que distancia pro hub + 2 km
hub alocado 11
hub_order 3
passou aqui1
passou aqui5
route_id 8
caiu aqui tb
14
    order_id  order_latitude  order_longitude                ordered_at_brt
13        14      -23.681077        -46.65347 2024-04-01 00:33:32.890790868
2024-04-01 00:33:32.890790868
tem rota aberta
dist order to order é  5.131018241298335 hub em questão 2
dist order to hub é  3.2489260243658324 hub em questão 2
distancia pro pedido menor que di

...

2024-04-04 00:11:00
não tem rota aberta
route_id 282
dist pro hub mais próximo é 11.07
hub alocado 16
882
     order_id  order_latitude  order_longitude      ordered_at_brt
881       882      -23.594323       -46.729679 2024-04-04 00:15:00
2024-04-04 00:15:00
tem rota aberta
dist order to order é  9.490807396820244 hub em questão 16
dist order to hub é  9.6003389783956 hub em questão 19
distancia pro pedido menor que distancia pro hub + 2 km
hub alocado 16 2
route_id 282
883
     order_id  order_latitude  order_longitude      ordered_at_brt
882       883      -23.567384       -46.608996 2024-04-04 00:20:00
2024-04-04 00:20:00
tem rota aberta
dist order to order é  7.7279216079227435 hub em questão 16
dist order to hub é  6.089144459941869 hub em questão 16
distancia pro pedido menor que distancia pro hub + 2 km
hub alocado 16 2
route_id 282
884
     order_id  order_latitude  order_longitude      ordered_at_brt
883       884      -23.710372        -46.64597 2024-04-04 00:26:00
2024-04-0

dist order to order é  0.8615246484505282 hub em questão 14
dist order to hub é  9.45230434927428 hub em questão 16
distancia pro pedido menor que distancia pro hub + 2 km
hub alocado 14 2
route_id 287
906
     order_id  order_latitude  order_longitude      ordered_at_brt
905       906      -23.746995        -46.62897 2024-04-04 02:08:00
2024-04-04 02:08:00
tem rota aberta
dist order to order é  8.436757563218029 hub em questão 4
dist order to hub é  5.231437862799061 hub em questão 3
distancia pro pedido maior que distancia pro hub + 2 km
hub alocado 3
hub_order 4
passou aqui1
passou aqui5
route_id 290
caiu aqui tb
907
     order_id  order_latitude  order_longitude      ordered_at_brt
906       907      -23.688445        -46.63451 2024-04-04 02:10:00
2024-04-04 02:10:00
tem rota aberta
dist order to order é  5.036045657403426 hub em questão 4
dist order to hub é  9.692850355007463 hub em questão 3
distancia pro pedido menor que distancia pro hub + 2 km
hub alocado 4 2
route_id 289
908

dist order to order é  7.378312707864984 hub em questão 16
dist order to hub é  12.656903359131435 hub em questão 14
distancia pro pedido menor que distancia pro hub + 2 km
hub alocado 16 2
route_id 296
928
     order_id  order_latitude  order_longitude      ordered_at_brt
927       928      -23.638653       -46.684818 2024-04-04 03:40:00
2024-04-04 03:40:00
tem rota aberta
dist order to order é  3.23879258854466 hub em questão 16
dist order to hub é  12.59104010687251 hub em questão 16
distancia pro pedido menor que distancia pro hub + 2 km
hub alocado 16 2
route_id 296
929
     order_id  order_latitude  order_longitude      ordered_at_brt
928       929      -23.646824       -46.637905 2024-04-04 03:44:00
2024-04-04 03:44:00
não tem rota aberta
route_id 297
dist pro hub mais próximo é 12.36
hub alocado 14
930
     order_id  order_latitude  order_longitude      ordered_at_brt
929       930      -23.620664       -46.669563 2024-04-04 03:47:00
2024-04-04 03:47:00
tem rota aberta
dist ord

2024-04-04 05:46:00
tem rota aberta
dist order to order é  7.049209349127003 hub em questão 3
dist order to hub é  11.286583465502428 hub em questão 14
distancia pro pedido menor que distancia pro hub + 2 km
hub alocado 3 2
route_id 304
955
     order_id  order_latitude  order_longitude      ordered_at_brt
954       955      -23.625654       -46.608367 2024-04-04 05:55:00
2024-04-04 05:55:00
tem rota aberta
dist order to order é  8.204141568323788 hub em questão 17
dist order to hub é  8.730742703072126 hub em questão 14
distancia pro pedido menor que distancia pro hub + 2 km
hub alocado 17 2
route_id 305
956
     order_id  order_latitude  order_longitude      ordered_at_brt
955       956      -23.636605       -46.624299 2024-04-04 06:01:00
2024-04-04 06:01:00
não tem rota aberta
route_id 306
dist pro hub mais próximo é 10.66
hub alocado 14
957
     order_id  order_latitude  order_longitude      ordered_at_brt
956       957      -23.603023        -46.61193 2024-04-04 06:05:00
2024-04-0

dist order to order é  13.837650294696918 hub em questão 16
dist order to hub é  3.898586384673261 hub em questão 3
distancia pro pedido maior que distancia pro hub + 2 km
hub alocado 3
hub_order 16
passou aqui1
passou aqui5
route_id 313
caiu aqui tb
979
     order_id  order_latitude  order_longitude      ordered_at_brt
978       979      -23.690755       -46.632647 2024-04-04 07:48:00
2024-04-04 07:48:00
tem rota aberta
dist order to order é  8.294010769901957 hub em questão 16
dist order to hub é  9.546422430057946 hub em questão 3
distancia pro pedido menor que distancia pro hub + 2 km
hub alocado 16 2
route_id 311
980
     order_id  order_latitude  order_longitude      ordered_at_brt
979       980      -23.611327       -46.640843 2024-04-04 07:51:00
2024-04-04 07:51:00
tem rota aberta
dist order to order é  10.299818793099444 hub em questão 7
dist order to hub é  9.174731438467525 hub em questão 16
distancia pro pedido menor que distancia pro hub + 2 km
hub alocado 7 2
route_id 312

In [347]:
order_allocated_hub['route_id'].nunique()

357

In [348]:
order_allocated_hub.groupby(['route_id'])['order_id'].count().sort_values()

route_id
1      1
357    1
23     1
22     1
29     1
      ..
327    4
36     4
35     4
34     4
2      4
Name: order_id, Length: 357, dtype: int64

In [349]:
order_allocated_hub

,route_id,order_id,hub_id
0,1,1,3
1,2,2,2
2,3,3,12
3,4,4,1
4,5,5,10
...,...,...,...
995,355,996,13
996,355,997,13
997,356,998,12
998,356,999,12


# Roteirização

In [350]:
order_allocated_hub = order_allocated_hub.rename(columns={'route_id':'route_id_new_model','hub_id':'hub_id_new_model'})
order_allocated_hub

,route_id_new_model,order_id,hub_id_new_model
0,1,1,3
1,2,2,2
2,3,3,12
3,4,4,1
4,5,5,10
...,...,...,...
995,355,996,13
996,355,997,13
997,356,998,12
998,356,999,12


In [351]:
df_routes = hubs_with_both_routes.merge(order_allocated_hub,on='order_id')
df_routes

,order_id,order_longitude,order_latitude,ordered_at_brt,hub_id,hub_longitude,hub_latitude,distance,route_id_moving_window,route_id_fixed_interval,route_id_new_model,hub_id_new_model
0,4,-46.718602,-23.695621,2024-04-01 00:06:39.311488463,1,-46.690592,-23.702916,2.97,1.0,1.0,4,1
1,9,-46.726169,-23.691987,2024-04-01 00:14:25.573482729,1,-46.690592,-23.702916,3.82,1.0,1.0,4,1
2,36,-46.740537,-23.730637,2024-04-01 01:18:32.589737090,1,-46.690592,-23.702916,5.95,2.0,2.0,17,1
3,43,-46.693407,-23.728043,2024-04-01 01:35:09.629815740,1,-46.690592,-23.702916,2.81,2.0,3.0,17,1
4,53,-46.686270,-23.714534,2024-04-01 02:03:21.638165969,1,-46.690592,-23.702916,1.36,3.0,4.0,24,1
...,...,...,...,...,...,...,...,...,...,...,...,...
995,894,-46.714932,-23.644153,2024-04-02 06:18:28.630844171,15,-46.704035,-23.662853,2.36,480.0,595.0,312,6
996,895,-46.701347,-23.640940,2024-04-02 06:18:36.772386743,15,-46.704035,-23.662853,2.45,480.0,595.0,312,6
997,906,-46.710908,-23.643755,2024-04-02 06:41:34.302114117,15,-46.704035,-23.662853,2.24,481.0,596.0,323,15
998,954,-46.707899,-23.674768,2024-04-02 08:08:58.442976286,15,-46.704035,-23.662853,1.38,482.0,597.0,339,15


In [352]:
n_routes = len(df_routes.drop_duplicates(subset=['route_id_new_model']))
route_id_counter = 1.0
fact_route_order = pd.DataFrame(columns=['route_id_new_model', 'order_id', 
                                        'order_id_delivery_order','route_total_distance',
                                        'order_partial_distance'])

In [353]:
for _ in range(n_routes):
    print(route_id_counter)
    one_route_df = filter_one_route(df_routes,'route_id_new_model','hub_id_new_model',n_routes,route_id_counter)
    distance_matrix = get_distance_matrix(one_route_df)
    tour, total_distance, partial_distances = solve_tsp_brute_force_without_return(distance_matrix)
    route_indexed_df = get_route_reindexed_df(one_route_df,'route_id_new_model',tour,total_distance,partial_distances)
    fact_route_order = pd.concat([fact_route_order,route_indexed_df],ignore_index=True)
    route_id_counter = route_id_counter+1

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0


C:\Users\EduardoMontez\AppData\Local\Temp\ipykernel_28552\1606161840.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  fact_route_order = pd.concat([fact_route_order,route_indexed_df],ignore_index=True)


16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
100.0
101.0
102.0
103.0
104.0
105.0
106.0
107.0
108.0
109.0
110.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
120.0
121.0
122.0
123.0
124.0
125.0
126.0
127.0
128.0
129.0
130.0
131.0
132.0
133.0
134.0
135.0
136.0
137.0
138.0
139.0
140.0
141.0
142.0
143.0
144.0
145.0
146.0
147.0
148.0
149.0
150.0
151.0
152.0
153.0
154.0
155.0
156.0
157.0
158.0
159.0
160.0
161.0
162.0
163.0
164.0
165.0
166.0
167.0
168.0
169.0
170.0
171.0
172.0
173.0
174.0
175.0
176.0
177.0
178.0
179.0
180.0
181.0
182.0
183.0
184.0
185.0
186.0
187.0
188.0
189.0
190.0
191.0
192.0
193.0
194.0
195.0
196.

In [354]:
fact_route_order_new = fact_route_order.copy()

In [355]:
fact_route_order_new['order_partial_distance'].sum()
#20099.992870707087


np.float64(2669.124065843453)

In [356]:
fact_route_order_new['route_id_new_model'].nunique()

357

In [357]:
fact_route_order_new

,route_id_new_model,order_id,order_id_delivery_order,route_total_distance,order_partial_distance
0,1,1,1,2.285645,2.285645
1,2,2,1,13.617698,5.170983
2,2,6,2,13.617698,1.891684
3,2,14,3,13.617698,3.248926
4,2,15,4,13.617698,3.306105
...,...,...,...,...,...
995,355,997,1,4.032969,2.311030
996,355,996,2,4.032969,1.721938
997,356,998,1,6.907182,1.755045
998,356,999,2,6.907182,5.152137


# Current models

In [358]:
import itertools

df_routes_current_fixed_window = df_routes.copy()
n_routes = len(df_routes_current_fixed_window.drop_duplicates(subset=['route_id_fixed_interval']))
route_id_counter = 1.0
fact_route_order = pd.DataFrame(columns=['route_id_fixed_interval', 'order_id', 
                                        'order_id_delivery_order','route_total_distance',
                                        'order_partial_distance'])

In [359]:
df_routes_current_fixed_window

,order_id,order_longitude,order_latitude,ordered_at_brt,hub_id,hub_longitude,hub_latitude,distance,route_id_moving_window,route_id_fixed_interval,route_id_new_model,hub_id_new_model
0,4,-46.718602,-23.695621,2024-04-01 00:06:39.311488463,1,-46.690592,-23.702916,2.97,1.0,1.0,4,1
1,9,-46.726169,-23.691987,2024-04-01 00:14:25.573482729,1,-46.690592,-23.702916,3.82,1.0,1.0,4,1
2,36,-46.740537,-23.730637,2024-04-01 01:18:32.589737090,1,-46.690592,-23.702916,5.95,2.0,2.0,17,1
3,43,-46.693407,-23.728043,2024-04-01 01:35:09.629815740,1,-46.690592,-23.702916,2.81,2.0,3.0,17,1
4,53,-46.686270,-23.714534,2024-04-01 02:03:21.638165969,1,-46.690592,-23.702916,1.36,3.0,4.0,24,1
...,...,...,...,...,...,...,...,...,...,...,...,...
995,894,-46.714932,-23.644153,2024-04-02 06:18:28.630844171,15,-46.704035,-23.662853,2.36,480.0,595.0,312,6
996,895,-46.701347,-23.640940,2024-04-02 06:18:36.772386743,15,-46.704035,-23.662853,2.45,480.0,595.0,312,6
997,906,-46.710908,-23.643755,2024-04-02 06:41:34.302114117,15,-46.704035,-23.662853,2.24,481.0,596.0,323,15
998,954,-46.707899,-23.674768,2024-04-02 08:08:58.442976286,15,-46.704035,-23.662853,1.38,482.0,597.0,339,15


In [360]:
for _ in range(n_routes):
    print(route_id_counter)
    one_route_df = filter_one_route(df_routes_current_fixed_window,'route_id_fixed_interval','hub_id',n_routes,route_id_counter)
    distance_matrix = get_distance_matrix(one_route_df)
    tour, total_distance, partial_distances = solve_tsp_brute_force_without_return(distance_matrix)
    route_indexed_df = get_route_reindexed_df(one_route_df,'route_id_fixed_interval',tour,total_distance,partial_distances)
    fact_route_order = pd.concat([fact_route_order,route_indexed_df],ignore_index=True)
    route_id_counter = route_id_counter+1

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0


C:\Users\EduardoMontez\AppData\Local\Temp\ipykernel_28552\2325915873.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  fact_route_order = pd.concat([fact_route_order,route_indexed_df],ignore_index=True)


22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
100.0
101.0
102.0
103.0
104.0
105.0
106.0
107.0
108.0
109.0
110.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
120.0
121.0
122.0
123.0
124.0
125.0
126.0
127.0
128.0
129.0
130.0
131.0
132.0
133.0
134.0
135.0
136.0
137.0
138.0
139.0
140.0
141.0
142.0
143.0
144.0
145.0
146.0
147.0
148.0
149.0
150.0
151.0
152.0
153.0
154.0
155.0
156.0
157.0
158.0
159.0
160.0
161.0
162.0
163.0
164.0
165.0
166.0
167.0
168.0
169.0
170.0
171.0
172.0
173.0
174.0
175.0
176.0
177.0
178.0
179.0
180.0
181.0
182.0
183.0
184.0
185.0
186.0
187.0
188.0
189.0
190.0
191.0
192.0
193.0
194.0
195.0
196.0
197.0
198.0
199.0
200.0
201.

In [361]:
fator_correct = 1.5
percorrido_real_velho = fact_route_order['order_partial_distance'].sum()*fator_correct
percorrido_real_novo = fact_route_order_new['order_partial_distance'].sum()*fator_correct
percorrido_real_velho,percorrido_real_novo

(np.float64(3616.043434291733), np.float64(4003.68609876518))

In [362]:
(percorrido_real_velho-percorrido_real_novo)/percorrido_real_velho

np.float64(-0.1072007766271131)

In [363]:
604*6+(6888.142774726873-559)*1.15+(1000-559)

11343.514190935903

In [364]:
365*6+(6996.80685275373-366)*1.15+(1000-366)

10449.42788066679

In [365]:
custo_rota = 6
custo_km_add = 1.15
custo_parada = 1

In [366]:
fact_route_order['route_id_fixed_interval'].nunique(),fact_route_order_new['route_id_new_model'].nunique()

(598, 357)

In [367]:
fact_route_order

,route_id_fixed_interval,order_id,order_id_delivery_order,route_total_distance,order_partial_distance
0,1.0,4,1,3.835033,2.965059
1,1.0,9,2,3.835033,0.869974
2,2.0,36,1,5.945934,5.945934
3,3.0,43,1,2.808672,2.808672
4,4.0,53,1,3.557170,1.364755
...,...,...,...,...,...
995,595.0,894,3,9.568991,2.357071
996,595.0,895,4,9.568991,1.429210
997,596.0,906,1,2.236091,2.236091
998,597.0,954,1,1.382060,1.382060


In [368]:
n_rotas_janela_fixa = fact_route_order['route_id_fixed_interval'].nunique()
n_rotas_novo_modelo = fact_route_order_new['route_id_new_model'].nunique()
n_pedidos = fact_route_order['order_id'].nunique()

In [369]:
custo_velho = custo_rota*n_rotas_janela_fixa+(percorrido_real_velho-n_rotas_janela_fixa)*custo_km_add+(n_pedidos-n_rotas_janela_fixa)*custo_parada
custo_novo = custo_rota*n_rotas_novo_modelo+(percorrido_real_novo-n_rotas_novo_modelo)*custo_km_add+(n_pedidos-n_rotas_novo_modelo)*custo_parada
custo_velho/1000,custo_novo/1000

(np.float64(7.460749949435493), np.float64(6.978689013579957))

In [373]:
(custo_novo-custo_velho)/custo_velho

np.float64(-0.06461293289852318)

In [374]:
unique_hubs['icon']='home'

In [375]:
unique_hubs.to_csv('hubs_simula.csv')

In [376]:
df_orders.to_csv('pedidos_simula.csv')